# Exp 04: Split large letter selectors 
Let's build upon the 


In [1]:
# standard libraries
from time import perf_counter_ns
import time

In [2]:
# external libraries
from matplotlib.patches import Rectangle
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import pandas as pd
import seaborn as sns

# custom libraries
from _run_constants import *
from part_00_file_db_utils import *
from part_00_process_functions import *

In [3]:
word_df, wg_df, letter_dict, char_matrix, \
    word_group_id_list, word_id_list, wchar_matrix = load_input_data(
        db_path=rc.DB_PATH, db_name=rc.DB_NAME,
        in_file_path=rc.IN_FILE_PATH)

...loading words into a dataframe...
...query execution took: 1.56 seconds...
...loading word groups into a dataframe...
...query execution took: 1.57 seconds...
...loading the letter dictionary...
...loading the char matrix...
...subsetting the char matrix...


In [4]:
wg_df.head()

,word,lcase,n_chars,first_letter,word_id,word_group_id,letter_group,letter_group_ranked,word_group_count
0,A,a,1,a,0,0,a,a,1
1,aa,aa,2,a,1,1,a,a,1
2,aal,aal,3,a,2,2,al,la,2
3,aalii,aalii,5,a,3,3,ail,lai,1
4,aam,aam,3,a,4,4,am,ma,2


In [5]:
# add a column to count records - because pandas is weird
wg_df['n_records'] = int(1)

In [6]:
wg_df.head()

,word,lcase,n_chars,first_letter,word_id,word_group_id,letter_group,letter_group_ranked,word_group_count,n_records
0,A,a,1,a,0,0,a,a,1,1
1,aa,aa,2,a,1,1,a,a,1,1
2,aal,aal,3,a,2,2,al,la,2,1
3,aalii,aalii,5,a,3,3,ail,lai,1,1
4,aam,aam,3,a,4,4,am,ma,2,1


In [7]:
col_names = ['letter_selector_mod', 'n_records']
ls_df = build_letter_selector_df(df = wg_df, ls_nchar=3,                          
                                 letter_selector_col_name='letter_selector_mod',
                                 letter_selector_id_col_name='letter_selector_mod_id')

In [8]:
# now, load the previously calculated search space counts
sql = 'select * from exp_02_search_space_size;'

In [9]:
tot_ls_df = query_db(sql = sql, db_path = rc.DB_PATH, db_name = rc.DB_NAME)
rename_cols = {'letter_selector_temp':'letter_selector',
               'letter_selector_temp_id':'letter_selector_id'}
tot_ls_df = tot_ls_df.rename(columns = rename_cols)

...query execution took: 5.66 seconds...


In [10]:
tot_ls_df.head()

,letter_selector,ls_count,ls_nchar_iter,ls_nchar,letter_selector_id,ls_index,n_search_space
0,a,4,1,1,0,[0],133001
1,b,24368,1,1,1,[1],35319
2,c,3921,1,1,2,[2],80811
3,d,16164,1,1,3,[3],56705
4,e,1,1,1,4,[4],145106


In [11]:
tot_ls_df['tot_comps'] = tot_ls_df['ls_count'] * tot_ls_df['n_search_space']
tot_ls_df['ls_nchar'] = tot_ls_df['letter_selector'].str.len()

In [12]:
# check to make sure the counts are correct
tot_ls_df.loc[tot_ls_df['ls_nchar_iter'] == 3, 'ls_nchar'].value_counts().sum()

np.int64(2387)

In [13]:
# c1
# the graph suggests that search spaces greater than 30K are the issue
tot_ls_df['n_search_space_cut'] = (tot_ls_df['n_search_space'] >= 40000).astype(int)

In [14]:
# let's select where ls_nchar_iter == 3
mod_ls_df = tot_ls_df.loc[(tot_ls_df['ls_nchar_iter']==3) &
                      (tot_ls_df['ls_nchar'] == 3), :].copy()

In [15]:
mod_ls_df['n_search_space_cut'].sum()
# so, do we need to split up half of the records?
# this will add how many records?

np.int64(29)

In [16]:
mod_ls_df.head()

,letter_selector,ls_count,ls_nchar_iter,ls_nchar,letter_selector_id,ls_index,n_search_space,tot_comps,n_search_space_cut
372,bae,2,3,3,5,"[1, 0, 4]",15662,31324,0
373,bai,6,3,3,6,"[1, 0, 8]",13453,80718,0
374,bca,7,3,3,7,"[1, 2, 0]",7893,55251,0
375,bci,2,3,3,8,"[1, 2, 8]",6966,13932,0
376,bcl,553,3,3,9,"[1, 2, 11]",6039,3339567,0


In [17]:
add_split = mod_ls_df.loc[mod_ls_df['n_search_space_cut'] == 1, 'letter_selector'].tolist()
add_split = set(add_split)
len(add_split)

29

In [18]:
# split large search spaces by an extra letter
def compute_letter_selector(lgr):
    ls = lgr[:3]
    if ls in add_split:
        ls = lgr[:4]
    return ls

In [19]:
wg_df.head()

,word,lcase,n_chars,first_letter,word_id,word_group_id,letter_group,letter_group_ranked,word_group_count,n_records,letter_selector_mod
0,A,a,1,a,0,0,a,a,1,1,a
1,aa,aa,2,a,1,1,a,a,1,1,a
2,aal,aal,3,a,2,2,al,la,2,1,la
3,aalii,aalii,5,a,3,3,ail,lai,1,1,lai
4,aam,aam,3,a,4,4,am,ma,2,1,ma


In [20]:
wg_df['letter_selector_mod'] = wg_df['letter_group_ranked'].map(compute_letter_selector)

In [21]:
# create a new letter_selector
mod_ls_df = build_letter_selector_df(df = wg_df,
                                     ls_nchar=3,
                                     letter_selector_col_name='letter_selector_mod',
                                     letter_selector_id_col_name='letter_selector_mod_id',
                                     create_letter_selector=False)
mod_ls_df.shape

(2403, 5)

In [22]:
mod_ls_df.head()

,letter_selector_mod,ls_count,ls_nchar_iter,ls_nchar,letter_selector_mod_id
0,a,2,3,1,0
1,ae,1,3,2,1
2,ai,1,3,2,2
3,b,1,3,1,3
4,ba,4,3,2,4


In [23]:
# function to return the index position of each letter
def get_ls_index(ls:str):
    return [letter_dict[l] for l in ls]

In [24]:
mod_ls_df['ls_index'] = mod_ls_df['letter_selector_mod'].map(get_ls_index)

In [25]:
mod_ls_df['ls_nchar'].value_counts()

ls_nchar
3    2250
2     111
1      26
4      16
Name: count, dtype: int64

In [26]:
# load the total number of anagrams
n_possible_anagrams = load_possible_anagrams(db_path=rc.DB_PATH,
                                             db_name=rc.DB_NAME)

...query execution took: 0.0 seconds...


In [27]:
mod_ls_df = mod_ls_df.reset_index(drop = True)

In [28]:
mod_ls_df.head()

,letter_selector_mod,ls_count,ls_nchar_iter,ls_nchar,letter_selector_mod_id,ls_index
0,a,2,3,1,0,[0]
1,ae,1,3,2,1,"[0, 4]"
2,ai,1,3,2,2,"[0, 8]"
3,b,1,3,1,3,[1]
4,ba,4,3,2,4,"[1, 0]"


In [29]:
wg_df.head()

,word,lcase,n_chars,first_letter,word_id,word_group_id,letter_group,letter_group_ranked,word_group_count,n_records,letter_selector_mod
0,A,a,1,a,0,0,a,a,1,1,a
1,aa,aa,2,a,1,1,a,a,1,1,a
2,aal,aal,3,a,2,2,al,la,2,1,la
3,aalii,aalii,5,a,3,3,ail,lai,1,1,lai
4,aam,aam,3,a,4,4,am,ma,2,1,ma


In [30]:
mod_ls_df.head()

,letter_selector_mod,ls_count,ls_nchar_iter,ls_nchar,letter_selector_mod_id,ls_index
0,a,2,3,1,0,[0]
1,ae,1,3,2,1,"[0, 4]"
2,ai,1,3,2,2,"[0, 8]"
3,b,1,3,1,3,[1]
4,ba,4,3,2,4,"[1, 0]"


In [31]:
col_names = ['letter_selector_mod', 'letter_selector_mod_id']
wg_df = pd.merge(left = wg_df, right = mod_ls_df[col_names])

In [32]:
wg_df.head()

,word,lcase,n_chars,first_letter,word_id,word_group_id,letter_group,letter_group_ranked,word_group_count,n_records,letter_selector_mod,letter_selector_mod_id
0,A,a,1,a,0,0,a,a,1,1,a,0
1,aa,aa,2,a,1,1,a,a,1,1,a,0
2,aal,aal,3,a,2,2,al,la,2,1,la,1081
3,aalii,aalii,5,a,3,3,ail,lai,1,1,lai,1083
4,aam,aam,3,a,4,4,am,ma,2,1,ma,1115


In [33]:
curr_ls_df = build_letter_selector_df(df = wg_df,
                                      ls_nchar=3,
                                      letter_selector_col_name='letter_selector',
                                      letter_selector_id_col_name='letter_selector_id')
curr_ls_df['ls_index'] = curr_ls_df['letter_selector'].map(get_ls_index)

In [34]:
curr_ls_df.head()

,letter_selector,ls_count,ls_nchar_iter,ls_nchar,letter_selector_id,ls_index
0,a,2,3,1,0,[0]
1,ae,1,3,2,1,"[0, 4]"
2,ai,1,3,2,2,"[0, 8]"
3,b,1,3,1,3,[1]
4,ba,4,3,2,4,"[1, 0]"


In [35]:
col_names = ['letter_selector', 'letter_selector_id']
wg_df = pd.merge(left = wg_df, right = curr_ls_df[col_names])

In [36]:
wg_df['letter_selector'].unique().shape

(2387,)

In [37]:
wg_df['letter_selector_mod'].unique().shape

(2403,)

In [38]:
n_possible_anagrams / 2

99421122.5

In [39]:
# specify which letter selector to use
use_existing = False

if use_existing:
    ls_df = curr_ls_df.copy()
    ls_id_col_name = 'letter_selector_id'
else:
    ls_df = mod_ls_df.copy()
    ls_id_col_name = 'letter_selector_mod_id'


# run it!
run_start_time=perf_counter_ns()
# create the output list
output_list = np.full(shape = (int(n_possible_anagrams / 2), 2), fill_value=-1,dtype=np.int64)
output_time_list = []

# start counting
anagram_pair_count = 0

for ls_row_id, ls_row in ls_df.iloc[:None].iterrows():    
    if ls_row_id % 100 == 0:
        print(ls_row_id)
    start_time = perf_counter_ns()
    
    # get letter selector id information    
    ls_id = ls_row[ls_id_col_name]
    ls_id_index = np.array(object  = ls_row['ls_index'])    
        
    ##
    # SUBSET THE wchar_matrix by column selector
    ##    
    outcome_indices = np.all(wchar_matrix[:, ls_id_index] >= 1, axis=1)
    
    # this is the sub-matrix from which to query
    ls_wchar_matrix = wchar_matrix[outcome_indices, :]
        
    # this is the list of word group ids that correspond to the word group ids
    # in the ls_wchar_matrix
    temp_wg_id_list = word_group_id_list[outcome_indices]
    
    # this is the number of word groups that meet certain criteria. 
    # for example, words that feature the letters: 'buc'    
    n_search_space = temp_wg_id_list.shape[0]        
    
    # the current list of words featuring the set of least common letters.
    # these are the words have the least common letters of 'bro'    
    curr_wg_id_list = wg_df.loc[wg_df[ls_id_col_name] == ls_id, 'word_group_id'].to_numpy()
     
    for i_curr_wg_id, curr_wg_id in enumerate(curr_wg_id_list):    
            
        # get different word group ids?
        temp_wg_id = np.where(temp_wg_id_list == curr_wg_id)[0][0]
        #print(curr_wg_id, temp_wg_id)
        
        outcome_word_id_list = temp_wg_id_list[np.all(a = (ls_wchar_matrix - ls_wchar_matrix[temp_wg_id, :]) >= 0, axis = 1)]        
                
        n_from_words = outcome_word_id_list.shape[0]
        
        if n_from_words > 0:
            outcome_word_id_list = format_output_list(outcome_word_id_list=outcome_word_id_list, wg_id=curr_wg_id)
                        
            # enumerate the from/parent words
            new_anagram_pair_count = anagram_pair_count + n_from_words
            
            output_list[anagram_pair_count:new_anagram_pair_count, :] = outcome_word_id_list

            # update the anagram pair count
            anagram_pair_count = new_anagram_pair_count

    curr_time = calc_time(time_start=start_time, round_digits=8)
    output_time_list.append([ls_id, n_search_space, curr_time])

print('...time to find parent/child word relationships')
time_proc = calc_time(time_start=run_start_time, round_digits=4)
compute_elapsed_time(seconds=time_proc)
print('...truncating output list...')
output_indices = np.all(output_list >= 0, axis=1)
output_list = output_list[output_indices,]
print(output_list.shape)
time_proc = calc_time(time_start=run_start_time, round_digits=4)
compute_elapsed_time(seconds=time_proc)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
...time to find parent/child word relationships
Hours: 0 | minutes: 2 | seconds: 52.2755
...truncating output list...
(73218235, 2)
Hours: 0 | minutes: 2 | seconds: 57.8825


In [40]:
# c2
# current technique: 2 minutes, 37 seconds
# modified technique, with 40K cutoff, up to four characters: 2 minutes, 2 seconds
# modified technique, with 35K cutoff, up to four characters: 2 minutes, 18 seconds
# modified technique, with 30K cutoff, up to four characters: 2 minutes, 18 seconds
# modified technique, with 40K cutoff, up to five characters: 2 minutes, 2 seconds

In [41]:
# approximately 19 seconds
from_word_counter = collections.Counter(output_list[:,1])
to_word_counter = collections.Counter(output_list[:,0])

In [42]:
# the number of from word groups: should be 26
print(from_word_counter[746]) # should be 26
print(to_word_counter[746]) # should be 329

26
329
